<a href="https://colab.research.google.com/github/harish-git63/Demo1/blob/main/Bike_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import ttkbootstrap as ttk
from ttkbootstrap.constants import *
from tkinter import messagebox, filedialog
from PIL import Image, ImageTk
import openpyxl
import os

# ------------------ DATABASE SETUP ------------------
def connect_db():
    conn = sqlite3.connect("bike_info.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS bikes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            brand TEXT,
            model TEXT,
            year INTEGER,
            price REAL
        )
    """)
    conn.commit()
    return conn

# ------------------ FUNCTIONS ------------------
def add_bike():
    brand = combo_brand.get()
    model = combo_model.get()
    year = combo_year.get()
    price = entry_price.get().replace(",", "")

    if not (brand and model and year and price):
        messagebox.showwarning("Missing Info", "Please fill all fields.")
        return

    try:
        cursor.execute("INSERT INTO bikes (brand, model, year, price) VALUES (?, ?, ?, ?)",
                       (brand, model, int(year), float(price)))
        conn.commit()
        show_bikes()
        clear_fields()
    except Exception as e:
        messagebox.showerror("Error", str(e))

def show_bikes():
    for i in tree.get_children():
        tree.delete(i)
    cursor.execute("SELECT * FROM bikes")
    for row in cursor.fetchall():
        tree.insert("", END, values=(row[0], row[1], row[2], row[3], f"{int(row[4]):,}"))

def clear_fields():
    combo_brand.set("")
    combo_model.set("")
    combo_year.set("")
    entry_price.delete(0, END)

def update_model_options(event):
    selected_brand = combo_brand.get()
    models = bike_models.get(selected_brand, [])
    combo_model.config(values=models)
    combo_model.set("")

def search_bikes():
    keyword = entry_search.get()
    query = "SELECT * FROM bikes WHERE brand LIKE ? OR model LIKE ? OR year LIKE ?"
    for i in tree.get_children():
        tree.delete(i)
    cursor.execute(query, (f"%{keyword}%", f"%{keyword}%", f"%{keyword}%"))
    for row in cursor.fetchall():
        tree.insert("", END, values=(row[0], row[1], row[2], row[3], f"{int(row[4]):,}"))

def export_to_excel():
    file_path = filedialog.asksaveasfilename(defaultextension=".xlsx")
    if not file_path:
        return
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.append(["ID", "Brand", "Model", "Year", "Price"])
    for child in tree.get_children():
        ws.append(tree.item(child)['values'])
    wb.save(file_path)
    messagebox.showinfo("Exported", "Bike data exported to Excel successfully.")

def toggle_theme():
    current = app.style.theme.name
    new_theme = "darkly" if current == "flatly" else "flatly"
    app.style.theme_use(new_theme)

def format_price_input(event):
    value = entry_price.get().replace(",", "")
    if value.isdigit():
        formatted = "{:,}".format(int(value))
        entry_price.delete(0, END)
        entry_price.insert(0, formatted)

def delete_bike():
    selected = tree.selection()
    if not selected:
        messagebox.showwarning("No Selection", "Please select a bike to delete.")
        return
    bike_id = tree.item(selected[0])['values'][0]
    cursor.execute("DELETE FROM bikes WHERE id = ?", (bike_id,))
    conn.commit()
    show_bikes()
    messagebox.showinfo("Deleted", "Bike deleted successfully.")

# ------------------ BIKE MODEL DATA ------------------
bike_models = {
    "Honda": ["Shine", "Unicorn", "Hornet", "CBR 250R", "Activa"],
    "Yamaha": ["R15", "MT-15", "FZ", "Fascino", "Ray ZR"],
    "Bajaj": ["Pulsar 150", "Pulsar NS200", "Avenger", "Dominar 400", "CT 100"],
    "TVS": ["Apache RTR 160", "Apache RTR 200", "Jupiter", "Ntorq", "Sport"],
    "Royal Enfield": ["Classic 350", "Bullet 350", "Himalayan", "Interceptor 650", "Meteor 350"],
    "Suzuki": ["Gixxer", "Access 125", "Burgman", "Hayabusa", "Intruder"],
    "KTM": ["Duke 200", "Duke 390", "RC 200", "RC 390", "Adventure 250"],
    "Hero": ["Splendor", "Passion Pro", "Xtreme 160R", "Glamour", "Maestro"]
}

# ------------------ UI SETUP ------------------
app = ttk.Window(themename="flatly")
app.title("Bike Information Manager")
app.geometry("1000x600")
app.resizable(False, False)

conn = connect_db()
cursor = conn.cursor()

# ------------------ Logo ------------------
try:
    logo_img = Image.open("bike_logo.png").resize((80, 80))
    logo = ImageTk.PhotoImage(logo_img)
    logo_label = ttk.Label(app, image=logo)
    logo_label.place(x=10, y=10)
except Exception as e:
    print("Logo not found or failed to load:", e)

# ------------------ Form Fields ------------------
ttk.Label(app, text="Brand:").place(x=120, y=20)
combo_brand = ttk.Combobox(app, values=list(bike_models.keys()), state="readonly")
combo_brand.place(x=180, y=20)
combo_brand.bind("<<ComboboxSelected>>", update_model_options)

ttk.Label(app, text="Model:").place(x=400, y=20)
combo_model = ttk.Combobox(app, state="readonly")
combo_model.place(x=470, y=20)

ttk.Label(app, text="Year:").place(x=120, y=60)
combo_year = ttk.Combobox(app, values=[str(y) for y in range(2000, 2026)], state="readonly")
combo_year.place(x=180, y=60)

ttk.Label(app, text="Price:").place(x=400, y=60)
entry_price = ttk.Entry(app)
entry_price.place(x=470, y=60)
entry_price.bind("<KeyRelease>", format_price_input)

# ------------------ Buttons ------------------
ttk.Button(app, text="Add Bike", command=add_bike, bootstyle="success").place(x=700, y=20)
ttk.Button(app, text="Export to Excel", command=export_to_excel, bootstyle="info").place(x=700, y=60)
ttk.Button(app, text="Toggle Theme", command=toggle_theme, bootstyle="secondary").place(x=850, y=60)
ttk.Button(app, text="Delete Selected Bike", command=delete_bike, bootstyle="danger").place(x=700, y=100)

# ------------------ Search ------------------
ttk.Label(app, text="Search:").place(x=20, y=120)
entry_search = ttk.Entry(app, width=30)
entry_search.place(x=90, y=120)
ttk.Button(app, text="Search", command=search_bikes, bootstyle="primary").place(x=320, y=115)

# ------------------ Table ------------------
tree = ttk.Treeview(app, columns=("ID", "Brand", "Model", "Year", "Price"), show="headings")
for col in ("ID", "Brand", "Model", "Year", "Price"):
    tree.heading(col, text=col)
    tree.column(col, anchor="center")
tree.place(x=20, y=170, width=960, height=400)

show_bikes()
app.mainloop()
